In [1]:
import secrets
import csv
import pandas as pd
# 生成5个160位的随机密钥
keys = [secrets.token_hex(20) for _ in range(5)]

# 指定保存到CSV文件的路径
csv_file = 'random_keys2.csv'

# 将密钥保存到CSV文件
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    for key in keys:
        writer.writerow([key])

print(f'已将随机密钥保存到 {csv_file}')

已将随机密钥保存到 random_keys2.csv


In [2]:
import csv

csv_file = 'random_keys2.csv'

# 读取CSV文件并以每一行为分割
with open(csv_file, 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        if row:  # 确保行不为空
            random_key = row[0]
            print(random_key)

ab54174c6cf5ecf672cab8b745642524fbfe0103
0959ad61d9c75b395051e74f04957410620d800d
e916ade5db69018ca62cea83a3e5029c9619bc95
4644744ed234be8effa905e08a53aad8317926a6
ee6870452d79849e3d9b713c10f3d5527af9fa8f


In [3]:
#测试，读入第一个文件数据
import os
from scipy.io import loadmat
import re
folder_path = "D:\\finger_data\\data\\data1"
mat_files = [file for file in os.listdir(folder_path) if file.endswith('.mat')]
sorted_mat_files = sorted(mat_files,key=lambda name: int(re.search(r'\d+', name).group()))

first_mat_file = sorted_mat_files[2]
mat_data = loadmat(os.path.join(folder_path, first_mat_file))
print(type(mat_data))

if 'Ftemplate' in mat_data:
        Ftemplate_data = mat_data['Ftemplate']
        print("Ftemplate的数据:")

        print(len(Ftemplate_data[0]))
        print(Ftemplate_data.shape,type(Ftemplate_data))
else:
    print("MAT文件中没有名为'Ftemplate'的变量。")

<class 'dict'>
Ftemplate的数据:
299
(1, 299) <class 'numpy.ndarray'>


In [4]:
import random
import numpy as np


def generate_hash_codes(d, p, m, k,seed,vector):

    # Generate random feature vector
    x=vector

    # Initialize hash codes
    hash_codes = np.zeros(m, dtype=int)
    # 设置随机数生成器种子 
    random.seed(seed)     
    # 产生置换序列
    permutations = [[random.sample(range(d), d) for _ in range(p)] for _ in range(m)]
    
    # Generate Hadamard products
    
    for i in range(m):
        hat_x = np.zeros((p, d))
        for l in range(p):
            hat_x[l] = x[permutations[i][l]]

        # 随机初始化ti的值，梦卡罗特排序
        t_i = np.random.randint(1, k + 1)

        # 计算 p 阶Hadamard乘积向量
        hadamard_product_vector = np.ones(d)
        for l in range(p):
            hadamard_product_vector *= hat_x[l]

        # Calculate Hadamard product value
        for j in range(1, k + 1):
            hadamard_product = hadamard_product_vector[j - 1]
            if hadamard_product > hadamard_product_vector[t_i - 1]:
                t_i = j

        hash_codes[i] = t_i

    return hash_codes


# Define your parameters
d = 100  # 特征向量大小
p = 2  # 哈德马积阶乘
m = 50  # m个哈德马积的结果
k = 3  # 窗口大小
seed = 41  # 设置种子
vector = Ftemplate_data[0]#numpy向量

# Call the function to generate hash codes
generated_hash_codes = generate_hash_codes(d, p, m, k,seed,vector)

# Print the generated hash codes
print("Generated Hash Codes:", generated_hash_codes)

Generated Hash Codes: [1 2 3 1 3 1 2 1 3 1 1 3 2 2 2 2 2 2 1 2 2 2 1 2 1 3 3 2 2 2 3 1 2 3 3 3 2
 1 1 1 3 3 3 3 2 1 3 1 2 2]


In [5]:
#根据100个人，一个手指5个采样，5个key生成的     模板
import hashlib
from scipy.io import loadmat
import numpy as np
import csv
import pandas as pd
m=200
csv_file = 'random_keys2.csv'
# 读取CSV文件并以每一行为分割
with open(csv_file, 'r') as file:
    for person in range(1, 101):
        for j in range(4, 9):
            fname = f'D:/finger_data/data/data1/{person}_{j}.mat'
            t = loadmat(fname)
            xn = t['Ftemplate']            
            row_number = 0  # 初始化文件行号计数器
            file.seek(0)  # 将文件指针移回文件的开头
            reader = csv.reader(file)  # 重新创建reader对象
            for row in reader:
                row_number += 1  # 增加文件行号计数器
                if row:  # 确保行不为空
                    random_key = int(hashlib.md5(row[0].encode()).hexdigest(), 16) % (2 ** 32)  # 对哈希值进行取模
                    np.random.seed(random_key)  # 设置随机数生成器的种子
                    x2 = np.random.randn(299, 100)
                    ji = np.dot(xn, x2)  # xn与x2相乘
                    transposed_ji=np.array(ji).reshape(100,)
                    result_ji=generate_hash_codes(100, 2, m, 50,41,transposed_ji)
                    getint = np.round(result_ji).astype(int).reshape(m,)  # 将结果取整并转换为整数类型并转置为(100,)格式
                    
                    file_name = f'D:/finger_data/URP_unlinkable/{person}_{j}_{row_number}.csv'  # 修改保存文件的命名方式
                    df = pd.DataFrame(getint)
                    df = df.transpose()  # 转置数据，使其横排而不是竖排
                    df.to_csv(file_name, index=False)
                    print(f"已保存模板: {person}_{j}_{row_number}.csv")

# 输出结果
print("数据处理完成")

已保存模板: 1_4_1.csv
已保存模板: 1_4_2.csv
已保存模板: 1_4_3.csv
已保存模板: 1_4_4.csv
已保存模板: 1_4_5.csv
已保存模板: 1_5_1.csv
已保存模板: 1_5_2.csv
已保存模板: 1_5_3.csv
已保存模板: 1_5_4.csv
已保存模板: 1_5_5.csv
已保存模板: 1_6_1.csv
已保存模板: 1_6_2.csv
已保存模板: 1_6_3.csv
已保存模板: 1_6_4.csv
已保存模板: 1_6_5.csv
已保存模板: 1_7_1.csv
已保存模板: 1_7_2.csv
已保存模板: 1_7_3.csv
已保存模板: 1_7_4.csv
已保存模板: 1_7_5.csv
已保存模板: 1_8_1.csv
已保存模板: 1_8_2.csv
已保存模板: 1_8_3.csv
已保存模板: 1_8_4.csv
已保存模板: 1_8_5.csv
已保存模板: 2_4_1.csv
已保存模板: 2_4_2.csv
已保存模板: 2_4_3.csv
已保存模板: 2_4_4.csv
已保存模板: 2_4_5.csv
已保存模板: 2_5_1.csv
已保存模板: 2_5_2.csv
已保存模板: 2_5_3.csv
已保存模板: 2_5_4.csv
已保存模板: 2_5_5.csv
已保存模板: 2_6_1.csv
已保存模板: 2_6_2.csv
已保存模板: 2_6_3.csv
已保存模板: 2_6_4.csv
已保存模板: 2_6_5.csv
已保存模板: 2_7_1.csv
已保存模板: 2_7_2.csv
已保存模板: 2_7_3.csv
已保存模板: 2_7_4.csv
已保存模板: 2_7_5.csv
已保存模板: 2_8_1.csv
已保存模板: 2_8_2.csv
已保存模板: 2_8_3.csv
已保存模板: 2_8_4.csv
已保存模板: 2_8_5.csv
已保存模板: 3_4_1.csv
已保存模板: 3_4_2.csv
已保存模板: 3_4_3.csv
已保存模板: 3_4_4.csv
已保存模板: 3_4_5.csv
已保存模板: 3_5_1.csv
已保存模板: 3_5_2.csv
已保存模板: 3_5_3.csv
已保存模板: 3_5_4.c

In [13]:
def match_biometric(query, template):  
    num_same = np.sum(query== template)
    return float (num_same/200 ) #设置

In [14]:
#matescore
#真匹配得分


import pandas as pd
import numpy as np


gescore = np.zeros((100, 200), dtype=float)

# 遍历所有指纹图像对
for i in range(1, 101):
    m = 0
    for j in range(4, 9):
        for k in range(1, 6):
            csv_file1 = f"D:/finger_data/URP_unlinkable/{i}_{j}_{k}.csv"
            for p in range(j + 1, 9):
                for q in range(1, 6):
                    if k != q:
                        csv_file2 = f"D:/finger_data/URP_unlinkable/{i}_{p}_{q}.csv"
                        df1 = pd.read_csv(csv_file1)
                        df2 = pd.read_csv(csv_file2)
                        all_data1 = df1.values
                        all_data2 = df2.values
                        gescore[i - 1, m] = match_biometric(all_data1, all_data2)
                        m += 1

# 保存gescore
gescore_file = "D:/finger_data/URP_matescore.csv"
gescore_df = pd.DataFrame(gescore)
gescore_df.to_csv(gescore_file, index=False, header=False)

KeyboardInterrupt: 

In [8]:
#nomatescore
#假匹配得分

match_scores = []
# 计算不同类之间的匹配得分
for j in range(4, 9):  # j的编号是5个，从4到8
    for k in range(1, 6):  # k的编号是5个，从1到5
        # j和k的编号相同的情况下，遍历所有不同的i
        for i in range(1, 101):  # i的编号是100个
            # 文件路径的生成方式
            csv_file1 = f"D:/finger_data/URP_unlinkable/{i}_{j}_{k}.csv"
            for p in range(i + 1, 101):  # i的编号是100个且不相同
                csv_file2 = f"D:/finger_data/URP_unlinkable/{p}_{j}_{k}.csv"
                df1 = pd.read_csv(csv_file1)
                df2 = pd.read_csv(csv_file2)
                all_data1 = df1.values
                all_data2 = df2.values
                # 匹配度量
                similarity = match_biometric(all_data1, all_data2)
                match_scores.append(similarity)


# 打印匹配得分矩阵
print("匹配得分矩阵：")
print(len(match_scores))

# 指定要保存的CSV文件路径
csv_file = "D:/finger_data/URP_nomate.csv"
matrix = np.array(match_scores).reshape(330, 375)
df = pd.DataFrame(matrix)
# 将数据框保存为 CSV 文件，不保存索引列
df.to_csv(csv_file, index=False,header=False)
print(f"数据已保存到 {csv_file}")

匹配得分矩阵：
123750
数据已保存到 D:/finger_data/URP_nomate.csv


In [9]:
#计算不可连接性
import csv
# 你的CSV文件名
csv_file1 = 'D:/finger_data/URP_matescore.csv'
csv_file2 = 'D:/finger_data/URP_nomate.csv'

# 用于存储数据的列表
data_list1 = []
data_list2 = []
for i, csv_file in enumerate([csv_file1, csv_file2], start=1):
    # 打开CSV文件并读取数据
    with open(csv_file, 'r') as file:
        csv_reader = csv.reader(file)   
        # 遍历每一行数据
        for row in csv_reader:
            # 将每一行数据放入列表
            for item in row:
                if i == 1:
                    data_list1.append(float(item))
                elif i == 2:
                    data_list2.append(float(item))
print(len(data_list1))
print(len(data_list2))

20000
123750


In [1]:
"""
# -*- coding:utf-8 -*-
@Project : 11.py
@File : unlink(0-1).py
@Author : xxxxxxxxxx
@Time : 2023/12/15 21:08

"""
import numpy
import numpy as np
import matplotlib
import pandas as pd
matplotlib.use('Agg')
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import pylab
import seaborn as sns   #version=0.9
import csv

def calculate_unlinkability(matedScoresFile, nonMatedScoresFile, figureFile, figureTitle='Unlinkability analysis',
                            legendLocation='upper right', omega=1.0, nBins=1000):
    # Evaluation code
    data_list1 = []
    data_list2 = []
    for i, csv_file in enumerate([matedScoresFile, nonMatedScoresFile], start=1):
        # 打开CSV文件并读取数据
        with open(csv_file, 'r') as file:
            csv_reader = csv.reader(file)
            # 遍历每一行数据
            for row in csv_reader:
                # 将每一行数据放入列表
                for item in row:
                    if i == 1:
                        data_list1.append(float(item))
                    elif i == 2:
                        data_list2.append(float(item))
    matedScores = np.array(data_list1)
    nonMatedScores = np.array(data_list2)

    if nBins == -1:
        nBins = min(len(matedScores) // 10, 100)

    # Define range of scores to compute D
    bin_edges = numpy.linspace(min([min(matedScores), min(nonMatedScores)]),
                               max([max(matedScores), max(nonMatedScores)]), num=nBins + 1, endpoint=True)
    bin_centers = (bin_edges[1:] + bin_edges[:-1]) / 2

    # Compute score distributions (normalized histogram)
    y1 = numpy.histogram(matedScores, bins=bin_edges, density=True)[0]
    y2 = numpy.histogram(nonMatedScores, bins=bin_edges, density=True)[0]

    # Compute LR and D
    LR = numpy.divide(y1, y2, out=numpy.ones_like(y1), where=y2 != 0)
    D = 2 * (omega * LR / (1 + omega * LR)) - 1
    D[omega * LR <= 1] = 0
    D[y2 == 0] = 0

    print(type(D))
    # Compute and print Dsys
    Dsys = numpy.trapz(x=bin_centers, y=D * y1)
    print("Dsys:",Dsys)

    # Plot final figure of D + score distributions
    plt.clf()

    sns.set_context("paper", font_scale=1.7, rc={"lines.linewidth": 2.5})
    sns.set_style("white")
    print(type(matedScores.shape))

     # 在numpy数组上进行多维索引操作
    ax = sns.kdeplot(matedScores,shade=False, label='Mated', color=sns.xkcd_rgb["medium green"])
    x1, y1 = ax.get_lines()[0].get_data()
    ax = sns.kdeplot(nonMatedScores, shade=False, label='Non-Mated', color=sns.xkcd_rgb["pale red"], linewidth=2.5,
                     linestyle='--')
    x2, y2 = ax.get_lines()[1].get_data()

    ax2 = ax.twinx()
    lns3, = ax2.plot(bin_centers, D, label='$\mathrm{D}_{\leftrightarrow}(s)$', color=sns.xkcd_rgb["denim blue"],
                     linewidth=5)


    ax.spines['top'].set_visible(False)
    ax.set_ylabel("Probability Density")
    ax.set_xlabel("Score")
    ax.set_title("%s, $\mathrm{D}_{\leftrightarrow}^{\mathit{sys}}$ = %.4f" % (figureTitle, Dsys), y=1.02)

    labs = [ax.get_lines()[0].get_label(), ax.get_lines()[1].get_label(), ax2.get_lines()[0].get_label()]
    lns = [ax.get_lines()[0], ax.get_lines()[1], lns3]
    ax.legend(lns, labs, loc=legendLocation)

    ax.set_ylim([0, max(max(y1), max(y2)) * 1.05])
    ax.set_xlim([bin_edges[0] * 0.98, bin_edges[-1] * 1.02])
    ax2.set_ylim([0, 1.1])
    ax2.set_ylabel("$\mathrm{D}_{\leftrightarrow}(s)$")

    plt.gcf().subplots_adjust(bottom=0.15)
    plt.gcf().subplots_adjust(left=0.15)
    plt.gcf().subplots_adjust(right=0.88)
    pylab.savefig(figureFile)
    plt.show()
if __name__ == '__main__':
    # Define file paths and other parameters
    # matedScoresFile = 'D:\\code3\\unlinkability-metric\\matedScores.hdf5'
    matedScoresFile = 'D:\\finger_data\\URP_matescore.csv'
    # nonMatedScoresFile = 'D:\\code3\\unlinkability-metric\\nonMatedScores.hdf5'
    nonMatedScoresFile = 'D:\\finger_data\\URP_nomate.csv'
    figureFile = '.\output_figure.png'
    figureTitle = 'Unlinkability analysis'
    legendLocation = 'upper right'
    omega = 1.0
    nBins = 100

    # 调用带有提供数据的函数
    calculate_unlinkability(matedScoresFile, nonMatedScoresFile, figureFile, figureTitle, legendLocation, omega, nBins)


<class 'numpy.ndarray'>
Dsys: 0.007985085838996243
<class 'tuple'>
